# Logistic regression in pytorch
## What's in this tutorial?
This notebook will walk you through the basic uses of `mandala` for storing
and tracking ML experiment results. It uses logistic regression on a synthetic
dataset as a "minimally interesting" example of a data management use case. By
following this ML mini-project, you will learn how to
- break up an experiment into Python functions whose calls can be
tracked and queried by `mandala`;
- use `mandala`'s memoization to avoid re-running expensive computations and to
naturally interact with and grow your project (by adjusting the parameters and/or
adding new code);
- repurpose the (pure Python) code of your experiments into a *query interface*
to their results "for free";
- modify, or create new versions of, your experimental primitives, and have them
  seamlessly interact with the results of previous runs.

Ultimatley, the features of `mandala` work together to enable you to evolve
complex ML projects by writing only the plain-Python code that you'd write in a
temporary in-memory interactive session, yet get the benefits of a
database-backed experiment tracking system. 

## Import libraries

In [1]:
from typing import Tuple
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset

# recommended way to import mandala functionality
from mandala.imports import *

# for reproducibility
np.random.seed(0)
torch.random.set_rng_state(torch.manual_seed(0).get_state())

/home/amakelov/workspace/current/conda_envs/mandala_3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define experiment primitives
You'll break the project into two main functions: to generate the synthetic
dataset, and to train the model. Below is fairly standard `pytorch` code for
these. Note the use of `@op` to mark the functions as tracked by `mandala` -
more on that shortly:

In [2]:
DATA_DIMENSION = 10

# main `mandala` decorator; like @functools.lru_cache, but with extra functionality.
# Currently, you must specify the exact number of inputs (i.e., no *args or **kwargs),
# and the number of outputs (using a type annotation with a `Tuple` if there are
# multiple outputs).
@op
def generate_dataset() -> Tuple[TensorDataset, TensorDataset]:
    """
    Generate a simple synthetic dataset for logistic regression, perform a
    80/20 train/test split, and return the results as `TensorDataset`s.
    """
    n_samples = 1000
    x = np.random.randn(n_samples, DATA_DIMENSION)
    y = x[:, 0] > 0
    x, y = torch.from_numpy(x).float(), torch.from_numpy(y).long()
    train_size = int(0.8 * n_samples)
    train_dataset = TensorDataset(x[:train_size], y[:train_size])
    test_dataset = TensorDataset(x[train_size:], y[train_size:])
    return train_dataset, test_dataset

In [3]:
class LogisticRegression(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(DATA_DIMENSION, 2)

    def forward(self, feature):
        output = self.linear(feature)
        return output


@op
def train_model(
    train_dataset: TensorDataset,
    test_dataset: TensorDataset,
    learning_rate: float = 0.001,
    batch_size: int = 100,
    num_epochs: int = 3,
) -> Tuple[LogisticRegression, float]:
    """
    Train a logistic model on the given training dataset with the given
    hyperparameters.

    Prints out the train loss and test accuracy at the end of
    each epoch. Returns the trained model and the final test accuracy.
    """
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LogisticRegression().to(device)
    loss = torch.nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        # train
        model.train()
        for xs, ys in train_loader:
            xs = xs.to(device)
            ys = ys.to(device)
            optimizer.zero_grad()
            output = model(xs)
            loss_value = loss(output, ys)
            loss_value.backward()
        optimizer.step()
        # test
        model.eval()
        accurate, total = 0, 0
        for xs, ys in test_loader:
            xs = xs.to(device)
            ys = ys.to(device)
            output = model(xs)
            _, predicted = torch.max(output.data, 1)
            total += ys.size(0)
            accurate += (predicted == ys).sum()
        acc = 100 * accurate / total
        print(
            f"Epoch: {epoch}, Training loss: {round(loss_value.item(), 2)}. Test accuracy: {round(acc.item(), 2)}"
        )
    return model, round(float(acc.item()), 2)

## "Hello world", or: run the pipeline and store the results
Now that you have defined the functions that make up your pipeline, you can
run it with the default parameters to see how well the model performs!

The `@op` decorator on the functions above tells `mandala` to track the calls to
these functions and store their results - but this only happens when you call
these functions *in the context of a given `Storage` object*. So go ahead and
create a storage for the project: 

In [4]:
storage = Storage()

This storage will hold the results of all the experiments you run in this
notebook. Now, run the pipeline and save its results by wrapping the code you'd
normally write in a `storage.run()` context manager:

In [5]:
with storage.run():
    train_dataset, test_dataset = generate_dataset()
    model, acc = train_model(train_dataset, test_dataset)
    print(f"Final accuracy: {acc}")

Epoch: 0, Training loss: 0.74. Test accuracy: 41.5
Epoch: 1, Training loss: 0.75. Test accuracy: 41.5
Epoch: 2, Training loss: 0.76. Test accuracy: 41.5
Final accuracy: ValueRef(41.5, uid=0b6...)


### What just happened?
A lot happened behind the scenes in these few lines of code! Let's break it
down:
- Inside the `storage.run()` block, each time an `@op`-decorated function is
called **for the first time** on a set of inputs, `mandala` stores the inputs
and outputs of this call in the storage. 
- Values shared between calls are stored only once. So
  `train_dataset` will appear in storage as both the output to the call to
  `generate_dataset`, and the input to the call to `train_model`.
- The `acc` object (like all objects returned by `@op`-decorated functions) is a
*value reference*, which is a value wrapped with storage-related metadata. 

So, what happens when you call `@op`-decorated functions *a second time* on the
same inputs? Find out by running the cell below:

In [6]:
with storage.run():
    train_dataset, test_dataset = generate_dataset()
    model, acc = train_model(train_dataset, test_dataset)
    print(f"Final accuracy: {acc}")

Final accuracy: ValueRef(41.5, uid=0b6...)


**Note that this time the intermediate training results did not get printed!**.
This is because `mandala` recognized that the inputs to the functions were the
same as before, and so it didn't need to re-run the calls.

### So what?
This was the simplest non-trivial use case of `mandala`! However, at this point
it is just a glorified `pickle`-based memoization system. Its real power comes
from the way in which `mandala`'s memoization *composes* with the rest of the
Python language, which allow you to manage complex experiments with the minimal
amount of plain-Python code, as we'll see next.

## Grow the project with new parameters
Running the pipeline once is nice, but where `mandala` really shines is in
enabling you to grow a computational project in various ways with the minimal
necessary code changes, and have the storage interfaces "just work". 

Let's begin exploring this by investigating the effect of changing the learning
rate of the model. So far, you have been using the default learning rate of
`0.001`. Let's try a few other values, but also see how they compare with the
default value. Thanks to memoization, this is easy to do without re-doing
expensive work: we can use a list of values for the `learning_rate` parameter
that includes the default, and compare:

In [7]:
with storage.run():
    train_dataset, test_dataset = generate_dataset()
    for learning_rate in [0.001, 0.01, 0.1]:
        model, acc = train_model(train_dataset, test_dataset, learning_rate)
        print(
            # `unwrap()` is used to get the value wrapped by a `ValueRef`
            f"===end of run=== learning_rate: {learning_rate}, acc: {round(unwrap(acc), 2)}"
        )

===end of run=== learning_rate: 0.001, acc: 41.5
Epoch: 0, Training loss: 0.81. Test accuracy: 61.5
Epoch: 1, Training loss: 0.73. Test accuracy: 61.5
Epoch: 2, Training loss: 0.71. Test accuracy: 61.5
===end of run=== learning_rate: 0.01, acc: 61.5
Epoch: 0, Training loss: 0.59. Test accuracy: 68.0
Epoch: 1, Training loss: 0.59. Test accuracy: 70.5
Epoch: 2, Training loss: 0.53. Test accuracy: 73.5
===end of run=== learning_rate: 0.1, acc: 73.5


Note that the first run was re-used from before, while the 2nd and 3rd were
freshly computed. We see that the higher the learning rate, the better the
final accuracy. Now, let's try varying the batch size as well:

In [8]:
with storage.run():
    train_dataset, test_dataset = generate_dataset()
    for batch_size in [100, 200, 400]:
        for learning_rate in [0.001, 0.01, 0.1]:
            model, acc = train_model(
                train_dataset, test_dataset, learning_rate, batch_size
            )
            print(
                f"===end of run=== batch_size: {batch_size}, learning_rate: {learning_rate}, acc: {round(unwrap(acc), 2)}"
            )

===end of run=== batch_size: 100, learning_rate: 0.001, acc: 41.5
===end of run=== batch_size: 100, learning_rate: 0.01, acc: 61.5
===end of run=== batch_size: 100, learning_rate: 0.1, acc: 73.5
Epoch: 0, Training loss: 0.84. Test accuracy: 39.5
Epoch: 1, Training loss: 0.88. Test accuracy: 39.5
Epoch: 2, Training loss: 0.87. Test accuracy: 40.0
===end of run=== batch_size: 200, learning_rate: 0.001, acc: 40.0
Epoch: 0, Training loss: 0.7. Test accuracy: 67.5
Epoch: 1, Training loss: 0.66. Test accuracy: 67.5
Epoch: 2, Training loss: 0.63. Test accuracy: 67.5
===end of run=== batch_size: 200, learning_rate: 0.01, acc: 67.5
Epoch: 0, Training loss: 0.8. Test accuracy: 56.0
Epoch: 1, Training loss: 0.78. Test accuracy: 57.5
Epoch: 2, Training loss: 0.71. Test accuracy: 62.0
===end of run=== batch_size: 200, learning_rate: 0.1, acc: 62.0
Epoch: 0, Training loss: 0.93. Test accuracy: 41.0
Epoch: 1, Training loss: 0.95. Test accuracy: 41.5
Epoch: 2, Training loss: 0.92. Test accuracy: 42.0


## Query the results
By now, you have run the pipeline with many different combinations of
parameters, and it's getting difficult to make sense of all the results so far.
One option to "query" the results is to just re-run the above workflow, or a
"sub-workflow" of it. 

For example, how might you get all the results for a given learning rate, e.g.
`learning_rate=0.1`? One answer: just by re-running the subset of the above code
using this value of the learning rate:

In [9]:
with storage.run():
    train_dataset, test_dataset = generate_dataset()
    for batch_size in [100, 200, 400]:
        for learning_rate in [0.1]:  # only change relative to previous cell
            model, acc = train_model(
                train_dataset, test_dataset, learning_rate, batch_size
            )
            print(
                f"===end of run=== batch_size: {batch_size}, learning_rate: {learning_rate}, acc: {round(unwrap(acc), 2)}"
            )

===end of run=== batch_size: 100, learning_rate: 0.1, acc: 73.5
===end of run=== batch_size: 200, learning_rate: 0.1, acc: 62.0
===end of run=== batch_size: 400, learning_rate: 0.1, acc: 50.5


This kind of storage access pattern is called **retracing**: you "retrace"
computational code that you have already run in order to recover the quantities
computed along the way. You can use retracing to query existing results (like
you did above), or to easily add new parameters/logic that need to compute over
existing results.

However, sometimes you don't have a specific piece of code to retrace and just
want to look at all the results in storage. For this, you can use a "full
storage search" query interface. One option for this is to directly point to
variables in the computation. For example, by referring to the `acc` variable
from the code above:

In [10]:
storage.similar(acc, context=True) # this may take a few seconds

Pattern-matching to the following computational graph (all constraints apply):
    a0 = Q() # input to computation; can match anything
    a1 = Q() # input to computation; can match anything
    train_dataset, test_dataset = generate_dataset()
    a2 = Q() # input to computation; can match anything
    _, acc = train_model(train_dataset=train_dataset, test_dataset=test_dataset, learning_rate=a1, batch_size=a2, num_epochs=a0)
    result = storage.df(a0, a1, test_dataset, train_dataset, a2, acc)


,a0,a1,test_dataset,train_dataset,a2,acc
6,3,0.001,"(([tensor(0.9978), tensor(0.2601), tensor(0.92...","(([tensor(1.7641), tensor(0.4002), tensor(0.97...",200,40.0
0,3,0.001,"(([tensor(0.9978), tensor(0.2601), tensor(0.92...","(([tensor(1.7641), tensor(0.4002), tensor(0.97...",100,41.5
3,3,0.001,"(([tensor(0.9978), tensor(0.2601), tensor(0.92...","(([tensor(1.7641), tensor(0.4002), tensor(0.97...",400,42.0
8,3,0.010,"(([tensor(0.9978), tensor(0.2601), tensor(0.92...","(([tensor(1.7641), tensor(0.4002), tensor(0.97...",400,50.5
4,3,0.100,"(([tensor(0.9978), tensor(0.2601), tensor(0.92...","(([tensor(1.7641), tensor(0.4002), tensor(0.97...",400,50.5
2,3,0.010,"(([tensor(0.9978), tensor(0.2601), tensor(0.92...","(([tensor(1.7641), tensor(0.4002), tensor(0.97...",100,61.5
5,3,0.100,"(([tensor(0.9978), tensor(0.2601), tensor(0.92...","(([tensor(1.7641), tensor(0.4002), tensor(0.97...",200,62.0
7,3,0.010,"(([tensor(0.9978), tensor(0.2601), tensor(0.92...","(([tensor(1.7641), tensor(0.4002), tensor(0.97...",200,67.5
1,3,0.100,"(([tensor(0.9978), tensor(0.2601), tensor(0.92...","(([tensor(1.7641), tensor(0.4002), tensor(0.97...",100,73.5


### What just happened? 🤯
Behind the scenes, `mandala` builds a computational graph that links the inputs
and outputs to each call (as well as elements of collections to the collection
itself, but this is a topic for another tutorial :) ). This means that the last
value of `acc` in the loop above "knows" that it was computed by a certain
composition of memoized functions. 

This composition serves as the "shape" of the query, which looks for
computations in the storage that follow this same pattern, but possibly with
different values. 

More formally,
- **variables** stand for a value stored in storage. These include the `Q()`
placeholder objects in the graph printed out above, as well as the values
returned by `@op`-decorated functions in the `storage.query()` block.
- **constraints** (i.e. function calls) enforce functional relationships between
variables.

The result of the query is a table, where each row is a choice of values for 
the variables that satisfy **all** the constraints.

If you want to limit the columns of the table, just omit the `context=True`
argument, which will restrict to only the variables you pass in explicitly:

In [11]:
storage.similar(acc)

Pattern-matching to the following computational graph (all constraints apply):
    a0 = Q() # input to computation; can match anything
    a1 = Q() # input to computation; can match anything
    train_dataset, test_dataset = generate_dataset()
    a2 = Q() # input to computation; can match anything
    _, acc = train_model(train_dataset=train_dataset, test_dataset=test_dataset, learning_rate=a1, batch_size=a2, num_epochs=a0)
    result = storage.df(acc)


,acc
3,40.0
1,41.5
7,42.0
5,50.5
6,50.5
4,61.5
0,62.0
8,67.5
2,73.5


Finally, you have the option of running or editing the query manually by
copy-pasting the graph into a `with storage.query()` context. For example, you
can give human-readable names to the variables:

In [12]:
with storage.query() as q:
    batch_size = Q() # input to computation; can match anything
    train_dataset, test_dataset = generate_dataset()
    learning_rate = Q() # input to computation; can match anything
    num_epochs = Q() # input to computation; can match anything
    _, acc = train_model(train_dataset=train_dataset, test_dataset=test_dataset, learning_rate=learning_rate, batch_size=batch_size, num_epochs=num_epochs)
storage.df(batch_size, learning_rate, num_epochs, acc)

Pattern-matching to the following computational graph (all constraints apply):
    num_epochs = Q() # input to computation; can match anything
    learning_rate = Q() # input to computation; can match anything
    train_dataset, test_dataset = generate_dataset()
    batch_size = Q() # input to computation; can match anything
    a0, acc = train_model(train_dataset=train_dataset, test_dataset=test_dataset, learning_rate=learning_rate, batch_size=batch_size, num_epochs=num_epochs)
    result = storage.df(batch_size, learning_rate, num_epochs, acc)


,batch_size,learning_rate,num_epochs,acc
6,200,0.001,3,40.0
0,100,0.001,3,41.5
3,400,0.001,3,42.0
8,400,0.010,3,50.5
4,400,0.100,3,50.5
2,100,0.010,3,61.5
5,200,0.100,3,62.0
7,200,0.010,3,67.5
1,100,0.100,3,73.5


## Try a larger dataset, or: how to modify memoized functions gracefully
A very common scenario in ML is to extend an existing experimental primitive
with new functionality, for example by exposing a hard-coded parameter, or
adding a new behavior to the function (e.g., an option to use a different
algorithm). With `mandala` you can seamlessly integrate this new functionality
with the results of previous runs.

For example, suppose you want to improve the accuracy numbers above. So far, you
have been running all models on a synthetic dataset of size `1000`. Let's try
increasing the dataset size while still sampling from the same distribution and
see if that helps! 

To do this, just directly modify the `generate_dataset` function
to take an additional argument `n_samples` with a default value of `1000`. Let's
also print out a message when it's called:

In [13]:
@op
def generate_dataset(n_samples: int = 1000) -> Tuple[TensorDataset, TensorDataset]:
    print("Hi from `generate_dataset` with a new argument!")
    x = np.random.randn(n_samples, DATA_DIMENSION)
    y = x[:, 0] > 0
    x, y = torch.from_numpy(x).float(), torch.from_numpy(y).long()
    train_size = int(0.8 * n_samples)
    train_dataset = TensorDataset(x[:train_size], y[:train_size])
    test_dataset = TensorDataset(x[train_size:], y[train_size:])
    return train_dataset, test_dataset

What happens to the memoized calls after we change the function? Find out by
re-running the project so far:

In [14]:
with storage.run():
    train_dataset, test_dataset = generate_dataset()
    for batch_size in [100, 200, 400]:
        for learning_rate in [0.001, 0.01, 0.1]:
            model, acc = train_model(
                train_dataset, test_dataset, learning_rate, batch_size
            )
            print(
                f"===end of run=== batch_size: {batch_size}, learning_rate: {learning_rate}, acc: {round(unwrap(acc), 2)}"
            )

===end of run=== batch_size: 100, learning_rate: 0.001, acc: 41.5
===end of run=== batch_size: 100, learning_rate: 0.01, acc: 61.5
===end of run=== batch_size: 100, learning_rate: 0.1, acc: 73.5
===end of run=== batch_size: 200, learning_rate: 0.001, acc: 40.0
===end of run=== batch_size: 200, learning_rate: 0.01, acc: 67.5
===end of run=== batch_size: 200, learning_rate: 0.1, acc: 62.0
===end of run=== batch_size: 400, learning_rate: 0.001, acc: 42.0
===end of run=== batch_size: 400, learning_rate: 0.01, acc: 50.5
===end of run=== batch_size: 400, learning_rate: 0.1, acc: 50.5


As you can see, `generate_dataset` was quiet, indicating that it was not re-run!
This is because `mandala` recognized that the value of `n_samples` was the
default, so it re-used the call to the previous (zero-argument)
`generate_dataset`. This is how adding new inputs to memoized functions works:
the default value (which must always be provided for new inputs) is used to
distinguish between old and new calls. You can add as many new inputs as you
want, as long as you provide a default value for each of them.

Let's finally run the pipeline with a larger dataset:

In [15]:
with storage.run():
    for n_samples in [1000, 2000]:
        train_dataset, test_dataset = generate_dataset(n_samples=n_samples)
        for batch_size in [100, 200, 400]:
            for learning_rate in [0.001, 0.01, 0.1]:
                model, acc = train_model(
                    train_dataset, test_dataset, learning_rate, batch_size
                )
                print(
                    f"===end of run=== n_samples: {n_samples}, batch_size: {batch_size}, learning_rate: {learning_rate}, acc: {round(unwrap(acc), 2)}"
                )

===end of run=== n_samples: 1000, batch_size: 100, learning_rate: 0.001, acc: 41.5
===end of run=== n_samples: 1000, batch_size: 100, learning_rate: 0.01, acc: 61.5
===end of run=== n_samples: 1000, batch_size: 100, learning_rate: 0.1, acc: 73.5
===end of run=== n_samples: 1000, batch_size: 200, learning_rate: 0.001, acc: 40.0
===end of run=== n_samples: 1000, batch_size: 200, learning_rate: 0.01, acc: 67.5
===end of run=== n_samples: 1000, batch_size: 200, learning_rate: 0.1, acc: 62.0
===end of run=== n_samples: 1000, batch_size: 400, learning_rate: 0.001, acc: 42.0
===end of run=== n_samples: 1000, batch_size: 400, learning_rate: 0.01, acc: 50.5
===end of run=== n_samples: 1000, batch_size: 400, learning_rate: 0.1, acc: 50.5
Hi from `generate_dataset` with a new argument!
Epoch: 0, Training loss: 0.85. Test accuracy: 51.5
Epoch: 1, Training loss: 0.93. Test accuracy: 51.5
Epoch: 2, Training loss: 0.78. Test accuracy: 51.5
===end of run=== n_samples: 2000, batch_size: 100, learning_r

Let's also get a nice table with the results; you can do this by minimally
modifying the query code from before!

**NOTE**: there is currently a bug in this query that causes the calls with
`n_samples=1000` not to show up.

In [16]:
with storage.query() as q:
    batch_size = Q() # input to computation; can match anything
    n_samples = Q() # input to computation; can match anything
    train_dataset, test_dataset = generate_dataset(n_samples=n_samples)
    learning_rate = Q() # input to computation; can match anything
    num_epochs = Q() # input to computation; can match anything
    _, acc = train_model(train_dataset=train_dataset, test_dataset=test_dataset, learning_rate=learning_rate, batch_size=batch_size, num_epochs=num_epochs)
storage.df(batch_size, learning_rate, num_epochs, n_samples, acc).sort_values(by='acc')

Pattern-matching to the following computational graph (all constraints apply):
    n_samples = Q() # input to computation; can match anything
    num_epochs = Q() # input to computation; can match anything
    learning_rate = Q() # input to computation; can match anything
    train_dataset, test_dataset = generate_dataset(n_samples=n_samples)
    batch_size = Q() # input to computation; can match anything
    a0, acc = train_model(train_dataset=train_dataset, test_dataset=test_dataset, learning_rate=learning_rate, batch_size=batch_size, num_epochs=num_epochs)
    result = storage.df(batch_size, learning_rate, num_epochs, n_samples, acc)


,batch_size,learning_rate,num_epochs,n_samples,acc
6,100,0.010,3,2000,33.00
3,100,0.001,3,2000,51.50
8,200,0.001,3,2000,55.00
7,200,0.010,3,2000,58.25
1,400,0.100,3,2000,58.50
0,400,0.001,3,2000,59.00
5,400,0.010,3,2000,69.75
4,200,0.100,3,2000,77.50
2,100,0.100,3,2000,78.00


This gives you a very simple way to co-evolve your computational code and the
query "interface" to its results!

However, it looks like the larger dataset didn't help! Why is that?

## "Oh no, there's a bug!", or: function versioning
If you look at the `train_model` function carefully, you'll see there's a
subtle but crucial mistake: because the call to `optimizer.step()` happens
outside the loop over the batches, the model is only updated once per epoch with
the last batch!

Unfortunately, we have already ran a lot of experiments with this bug. How can
we fix this without messing up our storage and re-running more than we need to?
Starting the whole project from scratch is not great, because it means we would
have to regenerate the datasets too. 

Fortunately, `mandala` has a simple way to deal with this situation with the
minimal possible amount of code changes. You must do two things:
- change the function to fix the bug, and **increment the
  version number**. By default, each function starts at version `0`. 
- re-run the experiments that used the old version of the function. All results
  that do not depend on the bug will be reused, and only the results that
  depend on values that the bugfix changes will be recomputed.

**NOTE**: there is an optional, more fine-grained versioning system that 
automatically tracks the dependencies of each memoized call for changes, and
allows you to effectively do the same as above (and much more), described in the
next tutorial.

So, fix the bug and increment the version number in the `@op` decorator:

In [17]:
@op(version=1)
def train_model(
    train_dataset: TensorDataset,
    test_dataset: TensorDataset,
    learning_rate: float = 0.001,
    batch_size: int = 100,
    num_epochs: int = 3,
) -> Tuple[LogisticRegression, float]:
    """
    Train a logistic model on the given training dataset with the given
    hyperparameters.

    Prints out the train loss and test accuracy at the end of
    each epoch. Returns the trained model and the final test accuracy.
    """
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LogisticRegression().to(device)
    loss = torch.nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        # train
        model.train()
        for xs, ys in train_loader:
            xs = xs.to(device)
            ys = ys.to(device)
            optimizer.zero_grad()
            output = model(xs)
            loss_value = loss(output, ys)
            loss_value.backward()
            optimizer.step()  #! bugfix here
        # test
        model.eval()
        accurate, total = 0, 0
        for xs, ys in test_loader:
            xs = xs.to(device)
            ys = ys.to(device)
            output = model(xs)
            _, predicted = torch.max(output.data, 1)
            total += ys.size(0)
            accurate += (predicted == ys).sum()
        acc = 100 * accurate / total
        print(
            f"Epoch: {epoch}, Training loss: {round(loss_value.item(), 2)}. Test accuracy: {round(acc.item(), 2)}"
        )
    return model, round(float(acc.item()), 2)

Now, literally copy-paste the computation + query code from above!

In [18]:
with storage.run():
    for n_samples in [1000, 2000]:
        train_dataset, test_dataset = generate_dataset(n_samples=n_samples)
        for batch_size in [100, 200, 400]:
            for learning_rate in [0.001, 0.01, 0.1]:
                model, acc = train_model(
                    train_dataset, test_dataset, learning_rate, batch_size
                )
                print(
                    f"===end of run=== n_samples: {n_samples}, batch_size: {batch_size}, learning_rate: {learning_rate}, acc: {round(unwrap(acc), 2)}"
                )

with storage.query() as q:
    batch_size = Q() # input to computation; can match anything
    n_samples = Q() # input to computation; can match anything
    train_dataset, test_dataset = generate_dataset(n_samples=n_samples)
    learning_rate = Q() # input to computation; can match anything
    num_epochs = Q() # input to computation; can match anything
    _, acc = train_model(train_dataset=train_dataset, test_dataset=test_dataset, learning_rate=learning_rate, batch_size=batch_size, num_epochs=num_epochs)
storage.df(batch_size, learning_rate, num_epochs, n_samples, acc).sort_values(by='acc')

Epoch: 0, Training loss: 0.95. Test accuracy: 38.5
Epoch: 1, Training loss: 0.88. Test accuracy: 38.5
Epoch: 2, Training loss: 0.88. Test accuracy: 38.5
===end of run=== n_samples: 1000, batch_size: 100, learning_rate: 0.001, acc: 38.5
Epoch: 0, Training loss: 0.84. Test accuracy: 54.0
Epoch: 1, Training loss: 0.75. Test accuracy: 56.5
Epoch: 2, Training loss: 0.73. Test accuracy: 57.0
===end of run=== n_samples: 1000, batch_size: 100, learning_rate: 0.01, acc: 57.0
Epoch: 0, Training loss: 0.64. Test accuracy: 69.5
Epoch: 1, Training loss: 0.46. Test accuracy: 87.0
Epoch: 2, Training loss: 0.36. Test accuracy: 92.5
===end of run=== n_samples: 1000, batch_size: 100, learning_rate: 0.1, acc: 92.5
Epoch: 0, Training loss: 0.71. Test accuracy: 55.5
Epoch: 1, Training loss: 0.73. Test accuracy: 55.5
Epoch: 2, Training loss: 0.71. Test accuracy: 55.5
===end of run=== n_samples: 1000, batch_size: 200, learning_rate: 0.001, acc: 55.5
Epoch: 0, Training loss: 0.78. Test accuracy: 53.5
Epoch: 1

,batch_size,learning_rate,num_epochs,n_samples,acc
6,400,0.010,3,2000,39.00
1,200,0.001,3,2000,39.25
8,100,0.001,3,2000,41.75
5,400,0.001,3,2000,50.75
4,100,0.010,3,2000,64.25
2,200,0.010,3,2000,72.75
3,400,0.100,3,2000,83.75
7,200,0.100,3,2000,93.00
0,100,0.100,3,2000,97.00


OK! Now we have the correct results. Maybe. At least, they make more sense than
before: accuracy went up, and the larger dataset actually helped.

## Compare model weights, or: track workflows with complex logic
So far, you have been running the same two-part pipeline independently for
several sets of parameters. This kind of workflow is common in ML projects, and
many libraries provide things like hyperparameter tuning interfaces for such
cases (e.g., `sklearn`). 

However, real-world projects often go beyond these "embarrassingly DAG-like"
workflows, and require more complex logic, which in turn leads to more complex
patterns for computation reuse and data management. 

This is another place where `mandala` shines: thanks to the transparent way you
build experiments directly in plain Python, you are free to make the logic as
complex as necessary, and still get a simple way to access the results.

To illustrate this, let's compare the trained model weights of models trained
with the same batch size and dataset, but different learning rates. First,
define a function that computes the distance between the weights of two models:

In [ ]:
@op
def get_model_distance(
    model_1: LogisticRegression,
    model_2: LogisticRegression,
) -> float:
    """
    Compute the distance between two models' weights
    """
    weight_distance = torch.dist(model_1.linear.weight, model_2.linear.weight)
    bias_distance = torch.dist(model_1.linear.bias, model_2.linear.bias)
    return round(float(weight_distance + bias_distance), 2)

Now, compare the mistakes of all the models trained on the synthetic dataset of
size `2000` for each batch size:

In [ ]:
with storage.run():
    train_dataset, test_dataset = generate_dataset(n_samples=2_000)
    for batch_size in [100, 200, 400]:
        for lr_1 in [0.001, 0.01, 0.1]:
            for lr_2 in [0.001, 0.01, 0.1]:
                if lr_1 <= lr_2:
                    continue
                model_1, acc_1 = train_model(
                    train_dataset, test_dataset, lr_1, batch_size
                )
                model_2, acc_2 = train_model(
                    train_dataset, test_dataset, lr_2, batch_size
                )
                model_dist = get_model_distance(model_1, model_2)
                print(
                    f"===end of run=== batch_size: {batch_size}, lr_1: {lr_1}, lr_2: {lr_2}, overlap coefficient: {unwrap(model_dist)}"
                )

Importantly, note that thanks to the memoization, the only new computations here
were calls to the new `compare_mistakes` function!

### Query the new workflow
How might you get a nice table with the results of the above workflow? Again,
you have two options. 

The simplest is just to retrace the code that you have already run, and collect 
the results as rows of a dataframe:

In [ ]:
with storage.run():
    rows = []
    train_dataset, test_dataset = generate_dataset(n_samples=2_000)
    for batch_size in [100, 200, 400]:
        for lr_1 in [0.001, 0.01, 0.1]:
            for lr_2 in [0.001, 0.01, 0.1]:
                if lr_1 <= lr_2:
                    continue
                model_1, acc_1 = train_model(
                    train_dataset, test_dataset, lr_1, batch_size
                )
                model_2, acc_2 = train_model(
                    train_dataset, test_dataset, lr_2, batch_size
                )
                model_distance = get_model_distance(model_1, model_2)
                rows.append(
                    {
                        "batch_size": batch_size,
                        "lr_1": lr_1,
                        "lr_2": lr_2,
                        "model_distance": unwrap(model_distance),
                    }
                )
df = pd.DataFrame(rows)
df.sort_values(by=["model_distance"], ascending=True)

Alternatively, you can also use the `storage.query()` context manager, but you
need to be careful to specify the correct constraints! In particular, you must
create two placeholders for the learning rate (why?). 

The easiest way to build the query structure is to copy-paste the computational
code, and then modify it to replace iteration over a parameter range with a
placeholder:

In [ ]:
with storage.query() as q:
    n_samples = Q().named("n_samples")
    train_dataset, test_dataset = generate_dataset(n_samples=n_samples)
    batch_size = Q().named("batch_size")
    lr_1 = Q().named("lr_1")
    lr_2 = Q().named("lr_2")
    model_1, acc_1 = train_model(train_dataset, test_dataset, lr_1, batch_size)
    model_2, acc_2 = train_model(train_dataset, test_dataset, lr_2, batch_size)
    model_distance = get_model_distance(model_1, model_2)
    df = q.get_table(batch_size, lr_1, lr_2, model_distance.named("model_distance"))

df.sort_values(by=["model_distance"], ascending=True)

You can see that more similar learning rates often (but not always) lead to a
smaller distance between the corresponding learned models!

## Conclusion
The project you just finished illustrated all the main features of
`mandala`, and how they work together to enable a very simple and direct
way of building and querying computational projects. While large-scale ML 
projects may require some additional patterns for properly using `mandala`, the
patterns you have already seen can take you a long way.

### The importance of a good function decomposition
Finally, it's a good time to reflect on something that often gets overlooked in
ML, but is especially important when it comes to effective data management with
`mandala`: **function decomposition**. 

Decomposing a project like the above into functions (i.e., `generate_dataset`,
`train_model`, `get_model_distance`) is something you often find yourself doing
almost without thinking. It's a necessary practice for managing complexity in
software engineering and ML experimentation in general. 

However, it is also particularly important to find a "good" decomposition when
tracking experiments with `mandala`, because the kind of decomposition you use
has consequences on the performance of your computations and storage system.

For example, consider an alternative decomposition with a single function that
does both the dataset generation and the model training:
```python
def generate_and_train(n_samples, batch_size, learning_rate) -> Tuple[LogisticRegression, float]:
    ...
```

A frequent use case is to train many models on the same dataset, but with
different hyperparameters. With a single function, you'd have to re-run the
dataset generation function every time, even though the dataset is the same!

This illustates the main principle of good function decomposition:
**functions should be as small as possible, but no smaller**. It's best to pack
independent units of work that you may want to combine in different ways into
separate functions.